In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class ResizeModel(nn.Module):
    def __init__(self, output=(224,224)):
        super(ResizeModel, self).__init__()

        self.output_size = output

    def forward(self, x):
        
        # 调整输入张量的大小
        x = F.interpolate(x, size=self.output_size, mode='bilinear', align_corners=False)

        return x

# 实例化模型
model = ResizeModel()

# 创建一个输入张量，注意这里的尺寸需要是模型可以接受的尺寸

dummy_input = torch.randn(1, 3, 1520, 2688)

# 导出模型为ONNX格式
onnx_file = "simple_resize_model.onnx"
torch.onnx.export(model,               # 模型要导出的对象
                  dummy_input,          # 模型输入的dummy数据
                  onnx_file,            # 输出的ONNX文件名
                  export_params=True,   # 存储训练好的参数权重
                  opset_version=11,     # ONNX操作符集版本
                  do_constant_folding=True, # 执行常量折叠优化
                  input_names = ['images'],   # 输入的名称
                  output_names = ['output0'],# 输出的名称
                #   dynamic_axes={'images' : {0 : 'batch_size'},    # 变量长度不定的维度名(Tensor的第二维开始)  
                #                 'output0' : {0 : 'batch_size'}}
                  )

print(f"Model exported to {onnx_file}")

In [ ]:
import onnxruntime as ort  
import numpy as np  
  
# 加载ONNX模型  
sess = ort.InferenceSession('simple_resize_model.onnx')  
  
# 获取输入和输出的名称  
input_name = sess.get_inputs()[0].name  
output_name = sess.get_outputs()[0].name  
  
# 准备输入数据（与训练时相同）  
dummy_input = np.random.rand(1, 3, 1520, 2688).astype(np.float32)  
  
# 进行推理  
ort_inputs = {input_name: dummy_input}  
ort_outputs = sess.run([output_name], ort_inputs)  
  
# 检查输出张量的尺寸  
print(ort_outputs[0].shape)  # 应该输出 (1, 3, 224, 224)

In [ ]:
import torch  
import torch.nn as nn  
import torch.onnx  
  
# 自定义一个包含reshape的层（尽管这不是常见的做法）  
class ReshapeLayer(nn.Module):  
    def __init__(self, *shape):  
        super(ReshapeLayer, self).__init__()  
        self.shape = shape  
  
    def forward(self, x):  
        return x.view(*self.shape)  
  
# 定义一个简单的网络模型  
class SimpleNetwork(nn.Module):  
    def __init__(self):  
        super(SimpleNetwork, self).__init__()  
        self.reshape_layer = ReshapeLayer(1, 3, 224, 224)  
  
    def forward(self, x):  
        x = self.reshape_layer(x)  
        # 在这里可以添加其他层，但为了简单起见，我们直接返回reshape后的Tensor  
        return x  
  
# 实例化网络模型  
net = SimpleNetwork()  
  
# 创建一个随机的输入Tensor，注意ONNX需要定义输入的形状  
dummy_input = torch.randn(1, 3, 1520, 2688)  
  
# 定义一个输出文件的名称  
onnx_file = 'simple_network.onnx'  
  
# 导出模型为ONNX格式  
# 注意：我们需要确保模型在评估模式下（即不启用dropout、batchnorm的training模式等）  
net.eval()  
with torch.no_grad():  # 不需要计算梯度  
    torch.onnx.export(net, dummy_input, onnx_file, verbose=True, input_names=['input_tensor'], output_names=['output_tensor'])  
  
print(f"Model exported to {onnx_file}")

In [4]:
import torch  
import torch.nn as nn  
import torch.onnx  
  
# 自定义一个包含reshape的层（尽管这不是常见的做法）  
class ReshapeLayer(nn.Module):  
    def __init__(self, *shape):  
        super(ReshapeLayer, self).__init__()  
        self.shape = shape  
  
    def forward(self, x):  
        return x.view(*self.shape)  
  
# 定义一个简单的网络模型  
class SimpleNetwork(nn.Module):  
    def __init__(self):  
        super(SimpleNetwork, self).__init__()  
        self.reshape_layer = ReshapeLayer(1, 3, 224, 224)  
  
    def forward(self, x):  
        x = self.reshape_layer(x)  
        # 在这里可以添加其他层，但为了简单起见，我们直接返回reshape后的Tensor  
        return x  
  
# 实例化网络模型  
net = SimpleNetwork()  
  
# 创建一个随机的输入Tensor，注意ONNX需要定义输入的形状  
dummy_input = torch.randn(1, 3, 1520, 2688)  
  
# 定义一个输出文件的名称  
onnx_file = 'simple_network.onnx'  
  
# 导出模型为ONNX格式  
# 注意：我们需要确保模型在评估模式下（即不启用dropout、batchnorm的training模式等）  
net.eval()  
with torch.no_grad():  # 不需要计算梯度  
    torch.onnx.export(net, dummy_input, onnx_file, verbose=True, input_names=['input_tensor'], output_names=['output_tensor'])  
  
print(f"Model exported to {onnx_file}")

RuntimeError: shape '[1, 3, 224, 224]' is invalid for input of size 12257280